BIO1 = Annual Mean Temperature  
BIO2 = Mean Diurnal Range (Mean of monthly (max temp - min temp))  
BIO3 = Isothermality (BIO2/BIO7) (×100)  
BIO4 = Temperature Seasonality (standard deviation ×100)  
BIO5 = Max Temperature of Warmest Month  
BIO6 = Min Temperature of Coldest Month  
BIO7 = Temperature Annual Range (BIO5-BIO6)  
BIO8 = Mean Temperature of Wettest Quarter  
BIO9 = Mean Temperature of Driest Quarter  
BIO10 = Mean Temperature of Warmest Quarter  
BIO11 = Mean Temperature of Coldest Quarter  
BIO12 = Annual Precipitation  
BIO13 = Precipitation of Wettest Month  
BIO14 = Precipitation of Driest Month  
BIO15 = Precipitation Seasonality (Coefficient of Variation)  
BIO16 = Precipitation of Wettest Quarter  
BIO17 = Precipitation of Driest Quarter  
BIO18 = Precipitation of Warmest Quarter  
BIO19 = Precipitation of Coldest Quarter

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import rasterio
from tqdm.auto import tqdm
import yaml

In [2]:
PATHS = [
    Path(f"wc2.1_30s/wc2.1_30s_bio_{i}.tif")
    for i in range(1, 19 + 1)
]

In [3]:
# Save the files with a better block shape

Path("wc2.1_30s_reformat/").mkdir(exist_ok=True)

for path in tqdm(PATHS):
    with rasterio.open(path) as dataset:
        path_out = Path("wc2.1_30s_reformat/") / path.name
        profile = dict(dataset.profile)
        profile.update(tiled=True, blockxsize=144, blockysize=80)
        with rasterio.open(path_out, "w", **profile) as dataset_out:
            dataset_out.write(dataset.read(masked=True))

  0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
# Check that the result is correct

for path in tqdm(PATHS):
    path_out = Path("wc2.1_30s_reformat/") / path.name
    with rasterio.open(path) as dataset, rasterio.open(path_out) as dataset2:
        assert (dataset.read() == dataset2.read()).all()

  0%|          | 0/19 [00:00<?, ?it/s]

In [5]:
# Calculate statistics

means = []
inv_stds = []
for path in tqdm(PATHS):
    path = Path("wc2.1_30s_reformat/") / path.name
    with rasterio.open(path) as dataset:
        data = dataset.read(masked=True)
        means.append(data.mean())
        inv_stds.append(1 / data.std())
        del data

  0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
print(repr(means))

[-4.444607781058798, 10.096086344492093, 34.370007727122236, 891.2090319373718, 13.787539469205488, -20.40271177134371, 34.19024379094415, -1.3611660708992686, -5.745056777226296, 6.941681041726127, -14.421294752932441, 532.131499794549, 91.45147736774582, 14.40531286690578, 75.82097431192203, 235.65372064235214, 52.30173682400658, 152.5487079282464, 104.11886525145661]


In [7]:
print(repr(inv_stds))

[0.040476010033391005, 0.3214225280748891, 0.05344555694549748, 0.0021399762797460935, 0.04606637955532394, 0.03843761811508031, 0.08331273603316934, 0.03430459261257326, 0.04390995464841662, 0.04717281544850558, 0.0373186472451054, 0.0015872099327125268, 0.009792140739007598, 0.036879766047647554, 0.02270102763233621, 0.0036459771924056417, 0.011045701305820062, 0.005456338375624682, 0.005517603053323615]
